In [43]:
%matplotlib tk
import numpy as np
import hyperspy.api as hs
import pyxem as pxm
import pymatgen as pmg
from matplotlib import pyplot as plt
from pymatgen.transformations.standard_transformations import RotationTransformation
from pyxem.indexation_generator import IndexationGenerator
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks
from scipy.constants import pi

In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
si = pmg.Element("Si")
lattice = pmg.Lattice.cubic(5.431)
silicon = pmg.Structure.from_spacegroup("Fd-3m",lattice, [si], [[0, 0, 0]])

ga = pmg.Element("Ga")
lattice = pmg.Lattice.hexagonal(1.531,5)
gall = pmg.Structure.from_spacegroup("P6_3mc",lattice, [ga], [[0, 0, 0]])

In [46]:
size = 256
radius=1.5
ediff = pxm.ElectronDiffractionCalculator(300., 0.025)

rotaxis = [0, 0, 1]
thetas = np.arange(0, 46, 45)

data_silicon = []
data_gall = []
for theta in thetas:
    rot = RotationTransformation(rotaxis, theta)
    sieg,garr = rot.apply_transformation(silicon),rot.apply_transformation(gall)
    diff_dat_s,diff_dat_g = ediff.calculate_ed_data(sieg, radius),ediff.calculate_ed_data(garr, radius)
    dpi_s     ,dpi_g      = diff_dat_s.as_signal(256, 0.05, 1.2), diff_dat_g.as_signal(256, 0.05, 1.2)
    data_silicon.append(dpi_s.data)
    data_gall.append(dpi_g.data)
    
data = [data_silicon] + [data_gall]
test_data = pxm.ElectronDiffraction(data)
test_data.set_calibration(1.2/128)

In [47]:
rot_list = []
nstep=119
for i in np.arange(nstep):
    theta = (i*59.5/(nstep-1))/180*pi
    rot_list.append((theta, 0., 0.))

In [48]:
edc = pxm.ElectronDiffractionCalculator(300, 0.025)
diff_gen = pxm.DiffractionLibraryGenerator(edc)
struc_lib = dict()
struc_lib['si'] = (silicon, rot_list)
struc_lib['ga'] = (gall,rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=1.5,
                                            representation='euler')

In [49]:
indexer = IndexationGenerator(test_data, library)
phase=["si","ga"] 
match_results = indexer.correlate(keys=phase)

A Jupyter Widget

In [50]:
peaks= match_results.map(peaks_from_best_template,phase=phase,library=library,inplace=False)

A Jupyter Widget

In [51]:
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
test_data.plot(cmap='viridis') 
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='red',marker='x')
    test_data.add_marker(m,plot_marker=True,permanent=True)